In [1]:
#!pip install transformers
#!pip install datasets
#!pip install huggingface_hub

In [2]:
#!pip install tensorflow


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager).
Your token has been saved to C:\Users\DAG9KOR\.cache\huggingface\token
Login successful


In [2]:
#!pip install git-lfs
!git config --global user.email "mruthyunjaya117@gmail.com"
!git config --global user.name "dhanunjaya"


In [38]:
!git init

Initialized empty Git repository in C:/Users/DAG9KOR/Downloads/.git/


In [40]:
#!git remote add  origin https://huggingface.co/dhanunjaya/distilgpt2-finetuned-medical

In [3]:
import transformers

print(transformers.__version__)

4.26.1


In [21]:
from datasets import load_dataset

#datasets = load_dataset("wikitext", "wikitext-2-raw-v1")
#C:\Users\DAG9KOR\Downloads\wikitext-2-raw-v1\wikitext-2-raw\wiki.train.raw

datasets1 = load_dataset("text", data_files={"train": 'pragmatic_train.txt',
                                                "validation": 'pragmatic_val.txt'})

Found cached dataset text (C:/Users/DAG9KOR/.cache/huggingface/datasets/text/default-f3a5ea111747c2d1/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


  0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
datasets1["train"][91]

{'text': 'Get Back to People.............................................................................34'}

In [23]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [24]:
show_random_elements(datasets1["train"])

,text
0,"tricked by the soldiers, who use the villagers' curiosity to get food from"
1,How's it going to sound to your boss?
2,Implementing DBC..........................................................................104
3,Keep Knowledge in Plain Text
4,Code That's Easy to Test.....................................................................165
5,•
6,tools to meet new challenges?
7,
8,resource allocation and deallocation. So let us suggest a simple tip:
9,"Select the date tab, and enter the date you noted for the Makefile in the first date field."


In [25]:
model_checkpoint = "distilgpt2"

In [26]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [27]:
print(tokenizer(datasets1["train"][10]['text']))

{'input_ids': [15167, 647, 6630, 832, 262, 3649, 43135, 290], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [28]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [29]:
tokenized_datasets = datasets1.map(
    tokenize_function, batched=True, num_proc=1, remove_columns=["text"]
)

Loading cached processed dataset at C:\Users\DAG9KOR\.cache\huggingface\datasets\text\default-f3a5ea111747c2d1\0.0.0\cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2\cache-a1ffbdc698834b44.arrow
Loading cached processed dataset at C:\Users\DAG9KOR\.cache\huggingface\datasets\text\default-f3a5ea111747c2d1\0.0.0\cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2\cache-3897399aa780e9f1.arrow


In [30]:
tokenized_datasets['train'][1]

{'input_ids': [], 'attention_mask': []}

In [31]:
block_size = 256

In [32]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, though you could add padding instead if the model supports it
    # In this, as in all things, we advise you to follow your heart
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [33]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=1,
)

Loading cached processed dataset at C:\Users\DAG9KOR\.cache\huggingface\datasets\text\default-f3a5ea111747c2d1\0.0.0\cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2\cache-c9f01e0d694d5bb3.arrow
Loading cached processed dataset at C:\Users\DAG9KOR\.cache\huggingface\datasets\text\default-f3a5ea111747c2d1\0.0.0\cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2\cache-d6688a96dc1d4e46.arrow


In [34]:
tokenizer.decode(lm_datasets["train"][2]["input_ids"])

"...................................................................23Challenges..........................................................................................23Stone Soup and Boiled Frogs................................................................23The Villagers' Side.............................................................................24Related sections include:................................................................25Challenges......................................................................................25Good-Enough Software.........................................................................25Involve Your Users in the Trade-Off................................................26Know When to Stop...........................................................................26Related sections include:................................................................27Challenges.........................................................................

In [35]:
from transformers import TFAutoModelForCausalLM

model = TFAutoModelForCausalLM.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [36]:
from transformers import create_optimizer, AdamWeightDecay

In [37]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [38]:
import tensorflow as tf

model.compile(optimizer=optimizer, jit_compile=True)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [39]:
train_set = model.prepare_tf_dataset(
    lm_datasets["train"],
    shuffle=True,
    batch_size=16,
)

validation_set = model.prepare_tf_dataset(
    lm_datasets["validation"],
    shuffle=False,
    batch_size=16,
)

In [ ]:
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard

model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = f"{model_name}-finetuned-wikitext2"

tensorboard_callback = TensorBoard(log_dir="./clm_model_save/logs")

push_to_hub_callback = PushToHubCallback(
    output_dir="./clm_model_save",
    tokenizer=tokenizer,
    #hub_model_id=push_to_hub_model_id
    hub_model_id="dhanunjaya/distilgpt2-finetuned-pragmatic-10",
)

callbacks = [tensorboard_callback, push_to_hub_callback]

model.fit(train_set, validation_data=validation_set, epochs=10,callbacks = callbacks)

Cloning https://huggingface.co/dhanunjaya/distilgpt2-finetuned-pragmatic-10 into local empty directory.


Epoch 1/10
19/19 [==============================] - 2635s 134s/step - loss: 4.2934 - val_loss: 4.1740
Epoch 2/10
14/19 [=====================>........] - ETA: 7:41 - loss: 4.1693

In [75]:
model.config.to_json_file("config.json")

In [74]:
eval_loss = model.evaluate(validation_set)

3/3 [==============================] - 11s 3s/step - loss: 4.7329


In [76]:
import math

print(f"Perplexity: {math.exp(eval_loss):.2f}")

Perplexity: 113.62


In [79]:
from transformers import AutoTokenizer, TFAutoModelForCausalLM

# You can, of course, use your own username and model name here 
# once you've pushed your model using the code above!
checkpoint = "dhanunjaya/distilgpt2-finetuned-wikitext2"
model = TFAutoModelForCausalLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at dhanunjaya/distilgpt2-finetuned-wikitext2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [80]:
test_sentence = "The classification of brain tumors"

In [83]:
tokenized = tokenizer(test_sentence, return_tensors="np")

outputs = model.generate(**tokenized, max_length=48)

print(outputs)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tf.Tensor(
[[  464 17923   286  3632 31155   318  1912   319   262  1271   286  4778
    287   262  3632   326   389  1944   287   262  3632   764   383  1271
    286  4778   287   262  3632   318  1912   319   262  1271   286  4778
    287   262  3632   326   389  1944   287   262  3632   764   383  1271]], shape=(1, 48), dtype=int32)


In [84]:
tokenizer.decode(outputs[0])

'The classification of brain tumors is based on the number of cells in the brain that are present in the brain. The number of cells in the brain is based on the number of cells in the brain that are present in the brain. The number'